<h1>Import Packages</h1>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import os

plt.style.use('fivethirtyeight')
sns.set()
plt.rcParams['figure.figsize'] = (12,12)

In [2]:
GRAPH = 'Results/Graphs'
TABLE = 'Results/Tables'
DATA = './data'
VERSION = 4  #1: Merged All 2:Cleaned
os.listdir(DATA)

['Dataset_v3.csv',
 'Dataset_v2.csv',
 'Jobs (2016-2018).xlsx',
 'ONET_Occupations.csv',
 'Information_Technology.csv',
 'Dataset_v1.csv',
 'Jobs (2019-2021) - Bdjobs.xlsx',
 'skills.csv',
 'Dataset_v4.csv',
 'ISCO.csv',
 'Skill_Rank.csv']

<h1>Load Pre-processed Data</h1>

In [12]:
df['TotalVacancy'].sum()/len(df)

2.3181593445401893

In [3]:
df = pd.read_csv(os.path.join(DATA,'Dataset_v'+str(VERSION) + '.csv'))
df['JobTitle'].nunique()

12550

<h1>Employment Type</h1>



In [14]:
emp_type_df = pd.DataFrame(df.groupby(['JobType'])['TotalVacancy'].sum()/df['TotalVacancy'].sum())
emp_type_df.rename(columns={'TotalVacancy':'mean'},inplace=True)
# emp_type_df['mean'] = df.groupby(['JobType'])['TotalVacancy'].mean()
emp_type_df['std'] = df.groupby(['JobType'])['TotalVacancy'].std()
emp_type_df['N'] = df.groupby(['JobType'])['TotalVacancy'].count()
emp_type_df.to_csv(os.path.join(TABLE,'emp_type.csv'),float_format='%.3f')
emp_type_df

,mean,std,N
JobType,,,
Contract,0.045368,11.232037,943
Freelance,0.008577,149.093137,11
FullTime,0.924163,7.385398,27060
Intern,0.010679,17.456123,138
PartTime,0.011197,11.863612,163


<h1>Position Level </h1>

In [15]:
pos_level_df = pd.DataFrame(df.groupby(['JobLevel'])['TotalVacancy'].sum()/df['TotalVacancy'].sum())
pos_level_df.rename(columns={'TotalVacancy':'mean'},inplace=True)
# pos_level_df['mean'] = df.groupby(['JobLevel'])['TotalVacancy'].mean()
pos_level_df['std'] = df.groupby(['JobLevel'])['TotalVacancy'].std()
pos_level_df['N'] = df.groupby(['JobLevel'])['TotalVacancy'].count()
pos_level_df.to_csv(os.path.join(TABLE,'pos_level.csv'),float_format='%.3f')
pos_level_df

,mean,std,N
JobLevel,,,
Entry,0.566963,11.797367,12854
Mid,0.405813,2.717020,14427
Top,0.027224,2.658081,1035


<h1>Gender</h1>

In [16]:
df['Gender'] = df['MaleGender'] + df['FemaleGender']
df['Gender'] = df['Gender'].apply(lambda x : 'Both male and female' if x == 2 else x)
df['Gender'] = df['Gender'].apply(lambda x : 'Not Specified' if x == 0 else x)
for i in range(len(df)):
    if df.loc[i,'Gender'] == 1:
        if df.loc[i,'MaleGender'] == 1:
            df.loc[i,'Gender'] = 'Male only'
        else:
            df.loc[i,'Gender'] = 'Female only'
            
df['Gender'].value_counts()

Both male and female    16889
Not Specified            7140
Male only                4068
Female only               219
Name: Gender, dtype: int64

In [17]:
gender_df = pd.DataFrame(df.groupby(['Gender'])['TotalVacancy'].sum()/df['TotalVacancy'].sum())
gender_df['N'] = df.groupby(['Gender'])['TotalVacancy'].count()
gender_df.to_csv(os.path.join(TABLE,'gender.csv'),float_format='%.3f')

<h1>Industry </h1>

In [4]:
top_industry = df['IndustryName'].value_counts().index[:9].tolist()
df['Industry'] = df['IndustryName'].apply(lambda x : x if x in top_industry else 'Others')

In [7]:
industry_df = pd.DataFrame(df.groupby(['Industry'])['TotalVacancy'].sum()/df['TotalVacancy'].sum())
industry_df.rename(columns={'TotalVacancy':'mean'},inplace=True)
# industry_df['mean'] = df.groupby(['Industry'])['TotalVacancy'].count()
# industry_df['std'] = df.groupby(['Industry'])['TotalVacancy'].std()
industry_df['N'] = df.groupby(['Industry'])['TotalVacancy'].count()
industry_df['mean']
# industry_df.to_csv(os.path.join(TABLE,'industry.csv'),float_format='%.3f')

Industry
Bank/ Non-Bank Fin. Institution    0.007313
Education                          0.026218
Garments/ Textile                  0.025883
Govt./ Semi-Govt./ Autonomous      0.001341
Information Technology (IT)        0.703280
Manufacturing (Heavy Industry)     0.011746
NGO/Development                    0.015280
Others                             0.148307
Telecommunication                  0.060633
Name: mean, dtype: float64

<h1>Location</h1>

In [20]:
df['Location_dhaka'] = df['City'].apply(lambda x : 'Dhaka' if x in ['Dhaka'] else None)
df['Location_bd'] = df['Country'].apply(lambda x : 'Dhaka or anywhere in Bangladesh' if x =='Bangladesh' else 'Outside Bangladesh')

In [21]:
location_df = pd.DataFrame(columns=['Location','Mean','N'])
location_df.loc[0] = ('Dhaka City',df[df['Location_dhaka'] == 'Dhaka']['TotalVacancy'].sum()/df['TotalVacancy'].sum(),
                     df[df['Location_dhaka'] == 'Dhaka']['TotalVacancy'].count())
location_df.loc[1] = ('Dhaka City or anywhere in Bangladesh',df[df['Location_bd'] == 'Dhaka or anywhere in Bangladesh']['TotalVacancy'].sum()/df['TotalVacancy'].sum(),
                     df[df['Location_bd'] == 'Dhaka or anywhere in Bangladesh']['TotalVacancy'].count())
location_df.loc[2] = ('Outside Bangladesh',df[df['Location_bd'] == 'Outside Bangladesh']['TotalVacancy'].sum()/df['TotalVacancy'].sum(),
                     df[df['Location_bd'] == 'Outside Bangladesh']['TotalVacancy'].count())
location_df.to_csv(os.path.join(TABLE,'location.csv'),float_format='%.3f')

In [99]:
df['Job_Responsibility']

0        Have to work at NIGHT SHIFT. It will be roaste...
1        Good understanding of OO Programming, OOAD con...
2        Leading the IT Division as Division Head & Con...
3        Provide Hardware, Software & ERP Related suppo...
4        Expert with Cisco network administration - CCN...
                               ...                        
28312    We are looking for an experienced data scienti...
28313    We are seeking a .Net Developer to join our te...
28314    Works directly with clients to support various...
28315    Develops Web look Intranet Portal Collaboratio...
28316    Design and develop advanced applications for t...
Name: Job_Responsibility, Length: 28317, dtype: object

In [24]:
industry_df

,mean,std,N
Industry,,,
Bank/ Non-Bank Fin. Institution,0.672269,1.152661,714
Education,1.693898,4.302800,1016
Garments/ Textile,1.232946,1.336658,1378
Govt./ Semi-Govt./ Autonomous,0.123596,0.448664,712
Information Technology (IT),2.857037,9.262398,16158
Manufacturing (Heavy Industry),0.982166,1.303327,785
NGO/Development,1.025562,1.564826,978
Others,1.891761,8.183402,5146
Telecommunication,2.785164,9.808854,1429
